In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import re

# Load the data
# From http://www.ssp.sp.gov.br/transparenciassp/Consulta.aspx SP Dados Criminais
data_csv = pd.read_csv('/content/drive/My Drive/crime stats sp/SPDadosCriminais_2023.csv')

In [ ]:
!pip install unidecode

In [ ]:
# Load the city data
# From http://blog.mds.gov.br/redesuas/lista-de-municipios-brasileiros/
city_data = pd.read_csv('/content/drive/My Drive/crime stats sp/Lista_Municípios_com_IBGE_Brasil_Versao_CSV.csv', delimiter=';', encoding='ISO-8859-1')

# Filter for cities in São Paulo state
sp_cities = city_data[city_data['UF'] == 'SP']['Município'].unique()

O plano era usar o nome das delegacias, consultar no bing e então adicionar o lat/lon. No fim, ficou dado demais pra mostrar e os pings ficariam grudados. Uma parte do código ficou inútil, incluindo o mapping de acrônimo, mas vou deixar porque se entendi bem uma parte das delegacias de S.Paulo são de outras cidades (eu não plotei, sempre dá OOM com muita coisa).

In [ ]:
# Define the mapping
acronym_mapping = {
    'DEL.': 'Delegacia ',
    'POL.': 'Policia ',
    'SEC.': 'Seccional ',
    'SEC': 'Seccional ',
    'DEL': 'Delegacia ',
    'POL': 'Policia ',
    'DDM': 'Delegacia Defesa da Mulher ',
    'D.P.': 'Delegacia de Policia ',
    'D.P': 'Delegacia de Policia ',
    'CPJ': 'Central de Policia Judiciaria ',
    'DHPP': 'Departamento de Homicídios e Proteção à Pessoa ',
    'DISE': 'Delegacia de Investigações Sobre Entorpecentes ',
    'DOPE': 'Departamento de Operações Policiais Estratégicas ',
    'DEIC': 'Departamento Estadual de Investigações Criminais ',
    'DIG': 'Delegacia de Investigações Gerais ',
    'SIG': 'Setor de Investigações Gerais ',
    'S.': 'SAO ',
    'PRES.': 'PRESIDENTE ',
    'SAO ANTONIO DO PINHAL': 'SANTO ANTONIO DO PINHAL ',
    'STA': 'SANTA ',
}


def replace_acronyms(string):
    for acronym, full_form in acronym_mapping.items():
        # Replace the acronym wherever it appears in the string
        string = re.sub(f"\\b{acronym}\\b", full_form, string)
    return string

data_csv['NOME_DELEGACIA_EXPANDED'] = data_csv['NOME_DELEGACIA'].dropna().apply(replace_acronyms)
# Replace any number of consecutive spaces with a single space in the 'NOME_DELEGACIA' column
data_csv['NOME_DELEGACIA_EXPANDED'] = data_csv['NOME_DELEGACIA_EXPANDED'].str.replace(r'\s+', ' ')


Como São Paulo é S.Paulo, ele não vai remover.

In [ ]:
from unidecode import unidecode

# Remove accents from city names
sp_cities_unaccented = [unidecode(city) for city in sp_cities]

# Concatenate all city names into a regex pattern, with word boundaries
city_pattern = r'\b(' + '|'.join(sp_cities_unaccented) + r')\b'

# Create a new column in the dataframe that is True if the unaccented police station name matches the city pattern
data_csv['Other_City'] = data_csv['NOME_DELEGACIA_EXPANDED'].apply(lambda x: unidecode(str(x))).str.contains(city_pattern, case=False, na=False)

# Filter the dataframe to only include rows where 'Other_City' is False
data_csv_filtered = data_csv[~data_csv['Other_City']]

# Drop the 'Other_City' column as it is no longer needed
data_csv_filtered = data_csv_filtered.drop(columns=['Other_City'])


In [ ]:
# Filter the data for São Paulo
data_sp = data_csv_filtered[data_csv_filtered['CIDADE'] == 'S.PAULO']

# Group by month, delegacia, and natureza_apurada (type of crime)
crime_stats = data_sp.groupby(['MES_ESTATISTICA', 'NOME_DELEGACIA_EXPANDED', 'NATUREZA_APURADA']).size().reset_index(name='COUNT')

# Display the result
print(crime_stats)

Só útil pra consulta no bing

In [ ]:
import requests

def get_geolocation(api_key, address):
    base_url = "http://dev.virtualearth.net/REST/v1/Locations"
    params = {
        "query": str(address) + ", São Paulo, Brazil",
        "key": api_key,
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['resourceSets'] and data['resourceSets'][0]['resources']:
            lat = data['resourceSets'][0]['resources'][0]['point']['coordinates'][0]
            lng = data['resourceSets'][0]['resources'][0]['point']['coordinates'][1]
            return lat, lng
    return None, None


In [ ]:
# Get the unique police station names
stations = data_sp['NOME_DELEGACIA_EXPANDED'].unique()

Caso queira voltar a consulta do bing, por agora deixei comentado

In [ ]:
# # Get the unique police station names
# stations = data_sp['NOME_DELEGACIA_EXPANDED'].unique()

# # Create a dictionary to store the results
# station_locations = {}

# # For each station
# for station in stations:
#     # Geocode the station
#     lat, lon = get_geolocation("api-key", station)
#     # Store the result
#     station_locations[station] = (lat, lon)


In [ ]:
# import numpy as np

# # If 'LATITUDE' or 'LONGITUDE' are null, replace with values from station_locations
# data_sp['LATITUDE_COMB'] = np.where(pd.isnull(data_sp['LATITUDE']),
#                                data_sp['NOME_DELEGACIA_EXPANDED'].map(lambda x: station_locations.get(x, (None, None))[0]),
#                                data_sp['LATITUDE'])

# data_sp['LONGITUDE_COMB'] = np.where(pd.isnull(data_sp['LONGITUDE']),
#                                 data_sp['NOME_DELEGACIA_EXPANDED'].map(lambda x: station_locations.get(x, (None, None))[1]),
#                                 data_sp['LONGITUDE'])


In [ ]:
!pip install keplergl

In [ ]:
# Create 'Latitude' and 'Longitude' columns from 'LATITUDE' and 'LONGITUDE'
data_sp['Latitude'] = data_sp['LATITUDE']
data_sp['Longitude'] = data_sp['LONGITUDE']


In [ ]:
data_sp_fil = data_sp[(data_sp['LATITUDE'] != 0) & (data_sp['LONGITUDE'] != 0)]

In [ ]:
# Specify the map center and zoom level
config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': -23.5505,
            'longitude': -46.6333,
            'zoom': 11
        }
    }
}

# Create a Kepler.gl map
map_1 = KeplerGl(height=500, config=config)

data_sample = data_sp_fil

data_subset = data_sample[['Latitude', 'Longitude', 'NATUREZA_APURADA']]

# Add the data to the map
map_1.add_data(data=data_subset, name="crimes")

In [ ]:
from IPython.display import Javascript
display(Javascript('''
  google.colab.widgets.installCustomManager('https://ssl.gstatic.com/colaboratory-static/widgets/colab-cdn-widget-manager/6a14374f468a145a/manager.min.js');
'''))
map_1

Salvar no drive, se quiser, no diretório crime stats sp

In [ ]:
map_1.save_to_html(file_name="/content/drive/MyDrive/crime stats sp/crime_map_sp.html")